In [1]:
## https://www.kaggle.com/code/anninasimon/pyspark-basics-for-beginners-part-1/notebook

In [2]:
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=748006361b764bc370ae42af3d138eef743c81c9b13e7362702675da97aff922
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/05 02:39:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.conf.set("spark.sql.session.timeZone", 'UTC+08:00')
## SET TIME ZONE 'America/Los_Angeles';

In [5]:
from pyspark.sql import functions

In [6]:
import time 

In [7]:
df=spark.read.csv("../input/userdata") 
df.schema.names

['_c0', '_c1', '_c2', '_c3', '_c4']

## 1. Data Cleaning 

In [8]:
## 1.1 column name 
df = df.toDF("user_id", "item_id", "category_id", "behavior_type", "input_timestamp")
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- behavior_type: string (nullable = true)
 |-- input_timestamp: string (nullable = true)



In [9]:
## 1.2 data format 
df = df.withColumn("tt", from_unixtime("input_timestamp"))
## Unix time - Epoch time - seconds passed since Epoch time 1970-01-01 00:00:00 UTC. 
## do not put show at the end 

NameError: name 'from_unixtime' is not defined

In [ ]:
df.printSchema()

In [ ]:
df.show(5)

In [ ]:
df = df.withColumn("d", to_date("tt")).withColumn("h", hour("tt"))
## to_date and hour 
## df.withColumn("hour", substring("timestamp", 0, 13)).show()

In [ ]:
df.show(5)

In [ ]:
## 1.2 Data Type

## from pyspark.sql.types import IntegerType, DateType
df = df.withColumn("d",col("d").cast("Date")).withColumn("h", hour("tt").cast("Integer"))

In [ ]:
df.printSchema()

In [ ]:
## 1.3 null value 
df.filter(df.user_id.isNull() & df.item_id.isNull() & df.category_id.isNull() \
          & df.category_id.isNull() & df.input_timestamp.isNull()).show()

In [ ]:

## 1.4 repeated value 
## df.groupBy("user_id", "item_id", "category_id", "behavior_type", "input_timestamp").count()
## %time : long running time 
## repeated with multiple devices 
## df.dropDuplicates()


In [ ]:
## df.count()
## 100150807

In [ ]:
## 1.5 anomaly 
## super long running time 
df_anomaly_lower = df.filter(df.d < '2017-11-25')
df_anomaly_higher = df.filter(df.d > '2017-12-03')

In [ ]:
## df_anomaly_lower.count()
## 53148

In [ ]:
## df_anomaly_higher.count()
## 2428

In [ ]:
higher_sample = df_anomaly_higher.sample(False, 0.1, seed=0)
higher_sample
## sample with replacement (default False).

In [ ]:
lower_sample = df_anomaly_lower.sample(False, 0.1, seed=0)
lower_sample

In [ ]:
df = df.filter((df.d > '2017-11-24') & (df.d < '2017-12-04') )
## df_check.count()
## 100095231

In [ ]:
## row_number(): only used on partitionBy
## df.withColumn("row_num", row_number().over(Window.partitionBy("Group") 
## groupBy vs partitionBy

## 2. Data Analysis

In [ ]:
## speed up pyspark 
## https://www.mikulskibartosz.name/how-to-speed-up-pyspark/
## spark.default.parallelism 

In [ ]:
df.createOrReplaceTempView("user_behavior")

In [ ]:
query ="SELECT \
	COUNT(DISTINCT(user_id)) AS user_count, \
FROM user_behavior;"
    ## COUNT(*) AS index,

In [ ]:
print(query)

In [ ]:
df01.show()
## df01
## long time running 

In [ ]:
df01.write.option("header",True).csv("./01_overview")
## super long

In [ ]:
query =" 
"

In [ ]:
df00 = spark.sql(query)

In [ ]:
df.write.option("header",True).csv("./name")